In [1]:
#importing the CSV file, original file can be found on GitHub page or at https://www.kaggle.com/datasets/sumitm004/forest-fire-area
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("forestfires.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [2]:
from calendar import month_abbr
#addidng a new column where month is displayed as the Int8 monthNum
lower_ma = [m.lower() for m in month_abbr]

df['monthNum'] = df['month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')
df.head(10)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,monthNum
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,8
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,8
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,8
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,9
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,9


In [3]:
#dropping the string values from the dataset as well as X and Y. X and Y were very specific to the Montesinho Natural Park and
#we want the model to be usable in any location
df = df.drop(columns=['X', 'Y','month','day'])
df.head(10)

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,monthNum
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,3
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,3
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,3
5,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,8
6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,8
7,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,8
8,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,9
9,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,9


In [4]:
#converting area to area_int as this is both simple and will work for the model

df['area_int'] = df['area'].astype({"area": int})
df = df.drop(columns=['area'])
df.head(10)

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,monthNum,area_int
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,3,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,10,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,10,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,3,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,3,0
5,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,8,0
6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,8,0
7,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,8,0
8,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,9,0
9,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,9,0


Cross Validation

In [5]:
#Kfold CV

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import cross_val_score


X, y = df.iloc[:,0:8 ], df.area_int
model=DecisionTreeClassifier()
kfold_validation=KFold(10)

mod_score=cross_val_score(model,X,y,cv=kfold_validation)
print(mod_score)
#Overall accuracy of the model will be average of all values.
print(np.mean(mod_score))

[0.5        0.28846154 0.61538462 0.01923077 0.13461538 0.28846154
 0.11538462 0.2745098  0.17647059 0.1372549 ]
0.25497737556561084


In [6]:
#Stratified kfold CV

from sklearn.model_selection import StratifiedKFold

sk_fold=StratifiedKFold(n_splits=10)

model=DecisionTreeClassifier()
mod_score2=cross_val_score(model,X,y,cv=sk_fold)
print(np.mean(mod_score2))
print(mod_score2)

c:\Users\krh52\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


0.23733031674208144
[0.28846154 0.25       0.38461538 0.19230769 0.44230769 0.28846154
 0.11538462 0.15686275 0.1372549  0.11764706]


In [7]:
#Repeated Random Test-Train Split

from sklearn.model_selection import ShuffleSplit

model=DecisionTreeClassifier()
s_split=ShuffleSplit(n_splits=10,test_size=0.3)
mod_score3=cross_val_score(model,X,y,cv=s_split)
print(mod_score3)
print(np.mean(mod_score3))

[0.28846154 0.3525641  0.29487179 0.28205128 0.24358974 0.29487179
 0.28205128 0.28846154 0.29487179 0.32692308]
0.2948717948717948


Using Scikit-learn SelectFromModel

In [8]:
#train test split

#split the dataset into train and test sets
from sklearn.model_selection import train_test_split

  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

In [9]:
#feature selection

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(DecisionTreeClassifier(),threshold='.4*mean')
sel.fit(X_train,y_train)

selected_feat= X_train.columns[(sel.get_support())]
print(selected_feat)

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind'], dtype='object')


In [10]:
#repeated random test-train split with selected features

from statistics import mean, stdev
from sklearn import preprocessing
                       
lst_accu_stratified = []
  
for train_index, test_index in s_split.split(X, y):
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    model.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(model.score(x_test_fold, y_test_fold))
  
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

List of possible accuracy: [0.2948717948717949, 0.3269230769230769, 0.26282051282051283, 0.27564102564102566, 0.33974358974358976, 0.32051282051282054, 0.32051282051282054, 0.2692307692307692, 0.30128205128205127, 0.30128205128205127]

Maximum Accuracy That can be obtained from this model is: 33.97435897435898 %

Minimum Accuracy: 26.282051282051285 %

Overall Accuracy: 30.128205128205128 %

Standard Deviation is: 0.025994711667941116
